In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import shutil
import requests
import os
import os.path
import re
import time
import requests
import pandas as pd

In [ ]:
driver = webdriver.Chrome(executable_path='./chromedriver.exe')
driver.get('https://www.imdb.com/chart/top/?ref_=nv_mv_250')
time.sleep(2)
detail_info = {}
parent_div = driver.find_element(By.XPATH,'//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul')
child_divs = parent_div.find_elements(By.XPATH,'./li')
urls = []
names = []
for child_div in child_divs:
    element = child_div.find_element(By.XPATH,'.//div[2]/div/div/div[1]/a')
    url = element.get_attribute('href')
    name = element.find_element(By.XPATH,'./h3').text
    rank, name = name.split('. ',1)
    urls.append(url)
    names.append(name)
movie_infos = pd.DataFrame(None,columns=['Rank','English_Title','URL','Directors','Writers','Stars'])

start = 0
problem_list = []
problem_No_list = []

In [ ]:
driver = webdriver.Chrome(executable_path='./chromedriver.exe')
for i in range(0,len(urls)):
    if i<start:
        continue
    
    url = urls[i]
    name = names[i]
    driver.get(url)
    time.sleep(2)
    main_director = ''
    directors = driver.find_element(By.XPATH,'/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[1]/div/ul')

    director_list = directors.find_elements(By.XPATH,'./li')
    for director in director_list:
        target_director = director.find_element(By.XPATH,'./a').text
        if len(main_director)==0:
            main_director = target_director
        else:
            main_director = main_director+','+target_director


    main_writer = ''
    writers = driver.find_element(By.XPATH,'/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[2]/div/ul')

    writer_list = writers.find_elements(By.XPATH,'./li')
    for writer in writer_list:
        target_writer = writer.find_element(By.XPATH,'./a').text
        if len(main_writer)==0:
            main_writer = target_writer
        else:
            main_writer = main_writer+','+target_writer
        
        
    try:
        main_star = ''
        stars = driver.find_element(By.XPATH,'/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]')

        star_list = stars.find_elements(By.XPATH,'./div')
        for star in star_list:
            target_star = star.find_element(By.XPATH,'./div[2]/a').text
            if len(main_star)==0:
                main_star = target_star
            else:
                main_star = main_star+','+target_star
        
        
    except KeyboardInterrupt:
        exit()
    
    except:
        main_star = []
        stars = driver.find_element(By.XPATH,'/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]/div[2]')
        star_list = stars.find_elements(By.XPATH,'./div')
        for star in star_list:
            target_star = star.find_element(By.XPATH,'./div[2]/a').text
            if len(main_star)==0:
                main_star = target_star
            else:
                main_star = main_star+','+target_star
            
            
    movie_info_dic = {
                        'Rank':i+1,
                        'English_Title':name,
                        'URL':url,
                        'Directors':main_director,
                        'Writers':main_writer,
                        'Stars':main_star}

    movie_info = pd.DataFrame.from_dict(movie_info_dic, orient='index').T

    movie_infos = pd.concat([movie_infos,movie_info],axis = 0,ignore_index=True)
    start = i+1

movie_infos.to_csv('./Top250_detailed_info.csv',index = False)